In [ ]:
!pip install accelerate
!pip install sentencepiece

In [19]:
#Converting .mp4 to wav file
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

def convert_video_to_audio(video_path, audio_path):
    try:
        video = VideoFileClip(video_path)
        audio = video.audio
        audio.write_audiofile(audio_path, codec='pcm_s16le')  # Saving as WAV format
        print("Audio file successfully created!")
    except Exception as e:
        print(f"Error: {e}")


video_path = "/content/ex1.mp4"  # TODO:Upgrade to support other codecs
file_name = video_path[9:-4]
audio_path = video_path[0:-4] + ".wav"
convert_video_to_audio(video_path, audio_path)

100%|██████████| 7.75M/7.75M [03:45<00:00, 74.7MiB/s]

MoviePy - Writing audio in /content/ex1.wav



chunk:  95%|█████████▍| 2223/2348 [00:01<00:00, 1982.05it/s, now=None]
100%|██████████| 7.75M/7.75M [03:47<00:00, 74.7MiB/s]

MoviePy - Done.
Audio file successfully created!


In [20]:
#Speech To Text
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model_for_speech_2_text = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model_for_speech_2_text.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model_for_speech_2_text,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

result = pipe(audio_path)
Transcript = result['text']
print(result)
print(result.keys())

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'text': " How to talk about hometown. I live in a city called London. This is not London, England. It is a city in Canada. Some of Canada's cities have the same names as English cities. London, Canada has many restaurants and cafes. For entertainment, you can go to the movie theatre or art gallery. There is a famous university in London called Western University. It is not close to the ocean or lake, so there are no beaches. It is a multicultural city so there are churches, temples and mosques. I've been living in London for 10 years. My favourite part of this city is the nature. There are many parks and forests where you can go for walks, runs, or have picnics with friends. The worst thing about London is that it is far away from other big cities in Canada. You need to drive for almost two hours to get to Toronto or Niagara Falls. This isolation isn't all bad, however. If you drive half an hour outside the city, you can find beautiful, quiet forests. Overall, I enjoy living in London

In [24]:
Transcript

" How to talk about hometown. I live in a city called London. This is not London, England. It is a city in Canada. Some of Canada's cities have the same names as English cities. London, Canada has many restaurants and cafes. For entertainment, you can go to the movie theatre or art gallery. There is a famous university in London called Western University. It is not close to the ocean or lake, so there are no beaches. It is a multicultural city so there are churches, temples and mosques. I've been living in London for 10 years. My favourite part of this city is the nature. There are many parks and forests where you can go for walks, runs, or have picnics with friends. The worst thing about London is that it is far away from other big cities in Canada. You need to drive for almost two hours to get to Toronto or Niagara Falls. This isolation isn't all bad, however. If you drive half an hour outside the city, you can find beautiful, quiet forests. Overall, I enjoy living in London."

In [21]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model_t = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer_t = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

Translated_Transcript = []

splited_transcript = Transcript.split(".")

for i in splited_transcript:
  tokenizer_t.src_lang = "en"
  encoded_hi = tokenizer_t(i, return_tensors="pt")
  generated_tokens = model_t.generate(**encoded_hi, forced_bos_token_id=tokenizer_t.get_lang_id("hi")).to(device)
  l = tokenizer_t.batch_decode(generated_tokens, skip_special_tokens=True)
  Translated_Transcript.append(l)

In [25]:
Translated_Transcript

[['घर के बारे में कैसे बात करें'],
 ['मैं लंदन नामक एक शहर में रहता हूं'],
 ['यह लंदन, इंग्लैंड नहीं है'],
 ['यह कनाडा में एक शहर है'],
 ['कनाडा के कुछ शहरों में अंग्रेजी शहरों के समान नाम हैं'],
 ['लंदन, कनाडा में कई रेस्तरां और कैफे हैं'],
 ['मनोरंजन के लिए, आप फिल्म थिएटर या कला गैलरी में जा सकते हैं'],
 ['लंदन में एक प्रसिद्ध विश्वविद्यालय है जिसे पश्चिमी विश्वविद्यालय कहा जाता है'],
 ['यह समुद्र या झील के करीब नहीं है, इसलिए कोई समुद्र तट नहीं है'],
 ['यह एक बहुसांस्कृतिक शहर है इसलिए चर्च, मंदिर और मस्जिदें हैं'],
 ['मैं दस साल से लंदन में रह रहा हूं।'],
 ['इस शहर का मेरा पसंदीदा हिस्सा प्रकृति है।'],
 ['कई पार्क और जंगल हैं जहां आप चलने, दौड़ने, या दोस्तों के साथ पिकनिक कर सकते हैं'],
 ['लंदन के बारे में सबसे बुरी बात यह है कि यह कनाडा के अन्य बड़े शहरों से बहुत दूर है'],
 ['आपको टोरंटो या नीआगारा झरने तक पहुंचने के लिए लगभग दो घंटे की ड्राइव की आवश्यकता है'],
 ['यह अलगाव सब बुरा नहीं है, हालांकि'],
 ['यदि आप शहर के बाहर आधे घंटे की ड्राइव करते हैं, तो आप सुंदर, शांत जंगल पा सकत

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [17]:
!pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 102.6 MB/s

In [26]:
single_string = ' '.join([sentence[0] for sentence in Translated_Transcript])

print(single_string)

घर के बारे में कैसे बात करें मैं लंदन नामक एक शहर में रहता हूं यह लंदन, इंग्लैंड नहीं है यह कनाडा में एक शहर है कनाडा के कुछ शहरों में अंग्रेजी शहरों के समान नाम हैं लंदन, कनाडा में कई रेस्तरां और कैफे हैं मनोरंजन के लिए, आप फिल्म थिएटर या कला गैलरी में जा सकते हैं लंदन में एक प्रसिद्ध विश्वविद्यालय है जिसे पश्चिमी विश्वविद्यालय कहा जाता है यह समुद्र या झील के करीब नहीं है, इसलिए कोई समुद्र तट नहीं है यह एक बहुसांस्कृतिक शहर है इसलिए चर्च, मंदिर और मस्जिदें हैं मैं दस साल से लंदन में रह रहा हूं। इस शहर का मेरा पसंदीदा हिस्सा प्रकृति है। कई पार्क और जंगल हैं जहां आप चलने, दौड़ने, या दोस्तों के साथ पिकनिक कर सकते हैं लंदन के बारे में सबसे बुरी बात यह है कि यह कनाडा के अन्य बड़े शहरों से बहुत दूर है आपको टोरंटो या नीआगारा झरने तक पहुंचने के लिए लगभग दो घंटे की ड्राइव की आवश्यकता है यह अलगाव सब बुरा नहीं है, हालांकि यदि आप शहर के बाहर आधे घंटे की ड्राइव करते हैं, तो आप सुंदर, शांत जंगल पा सकते हैं मैं लंदन में रहना पसंद करता हूं। के लिए


In [ ]:
from TTS.api import TTS

tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2")
tts.to(device)


text_translated = single_string
curr_file_path = "/content/output1.wav"
tts.tts_to_file(text=text_translated,
              file_path=curr_file_path,
              speaker_wav = "/content/ex1.wav",
              language="hi")

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

# Load the original video
original_video = VideoFileClip(video_path)

# Load the new audio file
new_audio = AudioFileClip("output1.wav")

# Replace the audio of the original video with the new audio
final_clip = original_video.set_audio(new_audio)

# Write the final video to a new file
final_clip.write_videofile("final_video.mp4")